# Introduction: Background of the Problem

It is often important for Law Enforcement Agency to identify the crime rate and pattern in an area to identify proper strategies: enforcement, education, campaigning etc. 

Chicago has violent crime about 49% higher than the average rate of crime in Illinois and 164% higher than the rest of the nation (https://www.areavibes.com/chicago-il/crime/). Therefore, it is often imortant for any pontetial home buyers or renters do some study and anlyze the crime data before making decisions of a safe enighborhood.

In this study we will inventigate a cluster analysis of the Chicago Neighborhood to find out the typical crime types, frequency etc. The analysis can also help someone a potential buyer to choose a relatively safe neighborhood. 

The study shows an application of web scraping, geo-coding and unsupervised machine learning technique (i.e. K-nearest neighborhood analysis) to analyze and cluster residential neighborhood in terms of types and extensity of crimes. The results can help a home buyer or renter choosing a safer neighborhood or cluster. It can also help law enforcement agencies consider strategic planning to mitigate the crimes.

The folowing two primary data sources were used for this study.
1. Neighborhood Information was availabel from the following wiki page: (https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago)

2. Crime information was Avialble from Chicago Data Portal
 (https://data.cityofchicago.org/Public-Safety/Crimes-Map/dfnk-7re6). The data was exported in csv format and imported into the Python dataframe for analysis. The file has all crimes for an year January 5, 2019 until Janaury 4, 2020
 
 The crime report shows the following salient fields:
 1. Case #
 2. Date of Occurence
 3. Approximate Block Address.
 4.Primary DEscription of the crime (i.e. BUrglary, Battery, Theft etc.)
 5. Secondary Description of the Crime
 6. Location Description
 7. Arrest made (Y/N)?
 8.Domestic Violence (Y/N)?
 9. Latitue and
 10. Longitude
 

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install lxml

In [3]:
import bs4 as bs
import urllib.request
#read the URL and clean by BeautifulSoup class
sauce = urllib.request.urlopen ('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago').read()
soup = bs.BeautifulSoup (sauce,'lxml')
#Check the title
soup.title

<title>List of neighborhoods in Chicago - Wikipedia</title>

In [4]:
#Read the table using find function since there is only one table available
table = soup.find('table')
#Read the table rows by reading tags 'tr'
table_rows =table.find_all('tr')
#Run loops to read the table rows by reading  < td > html tags
for tr in table_rows:
    td = tr.find_all('td')
    row= [i.text for i in td]
    print (row)

[]
['Albany Park\n', 'Albany Park\n']
['Altgeld Gardens\n', 'Riverdale\n']
['Andersonville\n', 'Edgewater\n']
['Archer Heights\n', 'Archer Heights\n']
['Armour Square\n', 'Armour Square\n']
['Ashburn\n', 'Ashburn\n']
['Ashburn Estates\n', 'Ashburn\n']
['Auburn Gresham\n', 'Auburn Gresham\n']
['Avalon Park\n', 'Avalon Park\n']
['Avondale\n', 'Avondale\n']
['Avondale Gardens\n', 'Irving Park\n']
['Back of the Yards\n', 'New City\n']
['Belmont Central\n', 'Belmont Cragin\n']
['Belmont Gardens\n', 'Hermosa\n']
['Belmont Heights\n', 'Dunning\n']
['Belmont Terrace\n', 'Dunning\n']
['Beverly\n', 'Beverly\n']
['Beverly View\n', 'Ashburn\n']
['Beverly Woods\n', 'Morgan Park\n']
['Big Oaks\n', 'Norwood Park\n']
['Boystown\n', 'Lake View\n']
['Bowmanville\n', 'Lincoln Square\n']
['Brainerd\n', 'Washington Heights\n']
['Brickyard\n', 'Belmont Cragin\n']
['Bridgeport\n', 'Bridgeport\n']
['Brighton Park\n', 'Brighton Park\n']
['Bronzeville\n', 'Douglas\n']
['Bucktown\n', 'Logan Square\n']
['Budlong 

In [5]:
#create a list of lists for all rowas
listrows=[]
j=0
for tr in table_rows:
    td = tr.find_all('td')
    row= [i.text for i in td]
    # exclude the header
    if (j>0):
        listrows.append(row)
    j=j+1
#Print first 10 rows    
listrows [:10]

[['Albany Park\n', 'Albany Park\n'],
 ['Altgeld Gardens\n', 'Riverdale\n'],
 ['Andersonville\n', 'Edgewater\n'],
 ['Archer Heights\n', 'Archer Heights\n'],
 ['Armour Square\n', 'Armour Square\n'],
 ['Ashburn\n', 'Ashburn\n'],
 ['Ashburn Estates\n', 'Ashburn\n'],
 ['Auburn Gresham\n', 'Auburn Gresham\n'],
 ['Avalon Park\n', 'Avalon Park\n'],
 ['Avondale\n', 'Avondale\n']]

In [6]:
# import pandas and convert the listrows into datafrane
import pandas as pd
df = pd.DataFrame(listrows)
df.head()

,0,1
0,Albany Park\n,Albany Park\n
1,Altgeld Gardens\n,Riverdale\n
2,Andersonville\n,Edgewater\n
3,Archer Heights\n,Archer Heights\n
4,Armour Square\n,Armour Square\n


In [7]:
#Create fucntion to clean the ending new line "\n"
def cleanNeighborhood(column):
    str2 = column.replace('\n', '')
    return str2

In [8]:
#Check the function before applying to the dataframe
print(cleanNeighborhood ('Albany Park\n'))

Albany Park


In [13]:
#assign header to the dataframe
df.columns = ['Neighborhood', 'Community Areas']
df.head()

,Neighborhood,Community Areas
0,Albany Park\n,Albany Park\n
1,Altgeld Gardens\n,Riverdale\n
2,Andersonville\n,Edgewater\n
3,Archer Heights\n,Archer Heights\n
4,Armour Square\n,Armour Square\n


In [15]:
#Apply the function to the dataframe
df['Neighborhood'] = df.apply(lambda row: cleanNeighborhood(row['Neighborhood']), axis=1)
df['Community Areas'] = df.apply(lambda row: cleanNeighborhood(row['Community Areas']), axis=1)
df.head()

,Neighborhood,Community Areas
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [18]:
#lets apple Geopy packate to get the info for Latlong
!pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Albany Park, Chicago ,IL")
print(location.latitude,location.longitude)

41.9719367 -87.7161739


In [28]:
#now, lets create a function that can return the Lat-Long
def get_geo_info(Neighborhood):
    from geopy.geocoders import Nominatim

    # Create geo_locator object instance
    geo_locator = Nominatim()
    place_name = Neighborhood+" , Chicago, IL"

    # Attempt to obtain geo data for given place name
    try:
        location = geo_locator.geocode(place_name)
    except Exception:
        return 0,0
    if not location:
        return 0,0
    return location.latitude,location.longitude 

In [30]:
df[['Latitude', 'Longitude']] = df.apply(lambda row: get_geo_info(row['Neighborhood']), axis=1,result_type='expand')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [39]:
#Lets check theeighborhood that could not be assigned a latitude and longitude
df[df['Latitude']==0]

,Neighborhood,Community Areas,Latitude,Longitude
6,Ashburn Estates,Ashburn,0.0,0.0
42,Cottage Grove Heights,Pullman,0.0,0.0
95,Irving Woods,Dunning,0.0,0.0
98,Jackson Park Highlands,South Shore,0.0,0.0
110,Lakewood / Balmoral,Edgewater,0.0,0.0
112,Legends South (Robert Taylor Homes),Grand Boulevard,0.0,0.0
125,Margate Park,Uptown,0.0,0.0
167,Polish Downtown,"West Town, Logan Square",0.0,0.0
176,Ranch Triangle,Lincoln Park,0.0,0.0
182,River's Edge,North Park,0.0,0.0


In [40]:
#let's remove these records from the analysis. 
#In the ideal cases the latitude and longitude can be manualy assigned by simple location search. 
#However, they are removed to simplify the process.
df =df[df['Latitude']!=0]
df.shape

(230, 4)

Therefore, in the final dataframe there are 230 neighborhood